# 예시 1: 동해 SST 장기 변화 분석

이 노트북은 동해 지역의 해수면 온도(SST) 장기 변화를 분석하는 예시입니다.
여러 데이터셋을 조합하여 시계열 분석, 공간 분석, 트렌드 분석을 수행합니다.

## 분석 목표
1. 동해 SST의 장기 트렌드 파악
2. 계절별 변동성 분석
3. 공간적 온도 변화 패턴 시각화
4. 극한 온도 이벤트 탐지

In [ ]:
# 환경 설정
import os
import sys
import warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats

sys.path.append('..')
import copernicus_utils as cu

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

print("분석 환경 준비 완료")

In [ ]:
# 동해 지역 설정
EAST_SEA_CONFIG = {
    'region_name': 'East Sea (Sea of Japan)',
    'lon_range': (128, 142),  # 경도 범위
    'lat_range': (35, 45),    # 위도 범위
    'analysis_period': {
        'start': '2000-01-01',
        'end': '2023-12-31'
    },
    'reference_period': {
        'start': '2000-01-01',
        'end': '2010-12-31'
    },
    'output_dir': '../output/east_sea_analysis/'
}

os.makedirs(EAST_SEA_CONFIG['output_dir'], exist_ok=True)
print(f"분석 지역: {EAST_SEA_CONFIG['region_name']}")
print(f"분석 기간: {EAST_SEA_CONFIG['analysis_period']['start']} ~ {EAST_SEA_CONFIG['analysis_period']['end']}")

## 1. 데이터 로딩 및 전처리

In [ ]:
# 샘플 데이터 생성 (실제 분석시 cu.load_dataset()으로 대체)
np.random.seed(42)

# 시간, 위도, 경도 좌표
time = pd.date_range(EAST_SEA_CONFIG['analysis_period']['start'], 
                     EAST_SEA_CONFIG['analysis_period']['end'], freq='D')
lat = np.arange(EAST_SEA_CONFIG['lat_range'][0], EAST_SEA_CONFIG['lat_range'][1], 0.25)
lon = np.arange(EAST_SEA_CONFIG['lon_range'][0], EAST_SEA_CONFIG['lon_range'][1], 0.25)

# 현실적인 SST 데이터 생성
sst_data = np.zeros((len(time), len(lat), len(lon)))

for t in range(len(time)):
    # 계절 변동 (여름: 25°C, 겨울: 10°C)
    day_of_year = time[t].dayofyear
    seasonal = 17.5 + 7.5 * np.sin(2 * np.pi * (day_of_year - 80) / 365.25)
    
    # 장기 온난화 트렌드 (24년간 1.2°C 상승)
    warming_trend = 0.05 * (t / 365.25)
    
    for i in range(len(lat)):
        for j in range(len(lon)):
            # 위도에 따른 온도 구배
            lat_effect = -(lat[i] - 35) * 0.5
            
            # 쿠로시오 해류 영향 (동쪽이 더 따뜻함)
            lon_effect = (lon[j] - 128) * 0.1
            
            # 랜덤 변동
            noise = np.random.randn() * 1.5
            
            sst_data[t, i, j] = seasonal + warming_trend + lat_effect + lon_effect + noise

# xarray Dataset 생성
ds = xr.Dataset(
    {
        'sst': (['time', 'latitude', 'longitude'], sst_data),
        'sst_uncertainty': (['time', 'latitude', 'longitude'], 
                           np.random.uniform(0.1, 0.3, sst_data.shape))
    },
    coords={
        'time': time,
        'latitude': lat,
        'longitude': lon
    },
    attrs={
        'title': 'East Sea SST Analysis Dataset',
        'source': 'Copernicus Marine Service',
        'processing_date': datetime.now().strftime('%Y-%m-%d')
    }
)

ds['sst'].attrs = {
    'units': '°C',
    'long_name': 'Sea Surface Temperature',
    'valid_min': -2.0,
    'valid_max': 35.0
}

print(f"데이터셋 생성 완료: {ds.dims}")

In [ ]:
# 데이터 품질 검사
qc = cu.quality_check(ds, 'sst', valid_range=(-2, 35), min_coverage=0.95)
print(f"\n데이터 품질 검사:")
print(f"  - 데이터 커버리지: {qc['coverage']:.1%}")
print(f"  - 유효 범위 내 데이터: {100 - qc.get('out_of_range_percent', 0):.1f}%")
print(f"  - 품질 평가: {'✓ 통과' if qc['quality_pass'] else '✗ 실패'}")

## 2. 시계열 분석

In [ ]:
# 공간 평균 시계열 생성
sst_ts = cu.create_timeseries(ds, 'sst', spatial_mean=True)

# 월별 평균 계산
monthly_ts = sst_ts.resample('M').mean()

# 연별 평균 계산
yearly_ts = sst_ts.resample('Y').mean()

print(f"시계열 데이터 생성:")
print(f"  - 일별: {len(sst_ts)} 데이터 포인트")
print(f"  - 월별: {len(monthly_ts)} 데이터 포인트")
print(f"  - 연별: {len(yearly_ts)} 데이터 포인트")

In [ ]:
# 트렌드 분석
trend_daily = cu.calculate_trend(sst_ts, method='linear')
trend_yearly = cu.calculate_trend(yearly_ts, method='linear')

# Sen's slope (비모수적 방법)
trend_sen = cu.calculate_trend(yearly_ts, method='sen')

print("\n트렌드 분석 결과:")
print(f"  일별 데이터 선형 트렌드:")
print(f"    - 기울기: {trend_daily['slope']*365.25:.4f}°C/year")
print(f"    - R²: {trend_daily['r_squared']:.3f}")
print(f"    - p-value: {trend_daily['p_value']:.4e}")
print(f"\n  연별 데이터 선형 트렌드:")
print(f"    - 기울기: {trend_yearly['slope']:.4f}°C/year")
print(f"    - R²: {trend_yearly['r_squared']:.3f}")
print(f"\n  Sen's slope: {trend_sen['slope']:.4f}°C/year")

In [ ]:
# 계절별 트렌드 분석
seasonal_trends = {}
seasons = {
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Autumn': [9, 10, 11],
    'Winter': [12, 1, 2]
}

for season_name, months in seasons.items():
    # 계절별 데이터 추출
    season_mask = sst_ts.index.month.isin(months)
    season_ts = sst_ts[season_mask]
    
    # 연평균 계산
    season_yearly = season_ts.resample('Y').mean()
    
    # 트렌드 계산
    if len(season_yearly) > 3:
        trend = cu.calculate_trend(season_yearly, method='linear')
        seasonal_trends[season_name] = trend

print("\n계절별 온난화 트렌드 (°C/year):")
for season, trend in seasonal_trends.items():
    print(f"  {season:6s}: {trend['slope']:+.4f} (R²={trend['r_squared']:.3f})")

## 3. 이상치 분석

In [ ]:
# 이상치 계산
anomaly = cu.calculate_anomaly(
    ds, 'sst',
    reference_period=(EAST_SEA_CONFIG['reference_period']['start'],
                     EAST_SEA_CONFIG['reference_period']['end'])
)

# 이상치 시계열
anomaly_ts = cu.create_timeseries(
    xr.Dataset({'sst_anomaly': anomaly}),
    'sst_anomaly',
    spatial_mean=True
)

print(f"이상치 통계:")
print(f"  - 평균: {anomaly_ts.mean():.3f}°C")
print(f"  - 표준편차: {anomaly_ts.std():.3f}°C")
print(f"  - 최대 양의 이상치: {anomaly_ts.max():.3f}°C")
print(f"  - 최대 음의 이상치: {anomaly_ts.min():.3f}°C")

In [ ]:
# 극한 온도 이벤트 탐지
# 상위 95 퍼센타일 (해양 열파)
heatwaves = cu.detect_extremes(
    sst_ts,
    threshold_type='percentile',
    threshold_value=95,
    duration=5  # 최소 5일 지속
)

# 하위 5 퍼센타일 (한파)
coldwaves = cu.detect_extremes(
    -sst_ts,  # 음수로 변환하여 낮은 온도 탐지
    threshold_type='percentile',
    threshold_value=95,
    duration=5
)

print(f"\n극한 이벤트 분석 (2000-2023):")
print(f"  해양 열파: {len(heatwaves)}개 이벤트")
if len(heatwaves) > 0:
    print(f"    - 평균 지속기간: {heatwaves['duration'].mean():.1f}일")
    print(f"    - 최장 지속기간: {heatwaves['duration'].max()}일")
    
print(f"\n  해양 한파: {len(coldwaves)}개 이벤트")
if len(coldwaves) > 0:
    print(f"    - 평균 지속기간: {coldwaves['duration'].mean():.1f}일")
    print(f"    - 최장 지속기간: {coldwaves['duration'].max()}일")

## 4. 공간 패턴 분석

In [ ]:
# 계절별 평균 공간 분포
seasonal_means = {}
for season_name, months in seasons.items():
    season_data = ds.sel(time=ds.time.dt.month.isin(months))
    seasonal_means[season_name] = season_data['sst'].mean('time')

# 온난화 패턴 (최근 5년 - 초기 5년)
early_period = ds.sel(time=slice('2000-01-01', '2004-12-31'))['sst'].mean('time')
recent_period = ds.sel(time=slice('2019-01-01', '2023-12-31'))['sst'].mean('time')
warming_pattern = recent_period - early_period

print("공간 패턴 분석 완료")

## 5. 종합 시각화

In [ ]:
# 종합 시계열 플롯
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(4, 2, figure=fig, hspace=0.3, wspace=0.2)

# 1. 일별 SST 시계열
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(sst_ts.index, sst_ts.values, color='lightblue', alpha=0.5, linewidth=0.5)
ax1.plot(monthly_ts.index, monthly_ts.values, color='blue', linewidth=1.5, label='Monthly mean')
ax1.plot(yearly_ts.index, yearly_ts.values, color='red', linewidth=2, label='Yearly mean')
ax1.plot(yearly_ts.index, trend_yearly['trend_line'].values, 
         color='darkred', linestyle='--', linewidth=2,
         label=f'Trend: {trend_yearly["slope"]:.3f}°C/year')
ax1.set_ylabel('SST (°C)')
ax1.set_title('East Sea SST Time Series (2000-2023)', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 2. 이상치 시계열
ax2 = fig.add_subplot(gs[1, :])
ax2.fill_between(anomaly_ts.index, 0, anomaly_ts.values,
                  where=(anomaly_ts.values > 0), color='red', alpha=0.5, label='Positive')
ax2.fill_between(anomaly_ts.index, 0, anomaly_ts.values,
                  where=(anomaly_ts.values <= 0), color='blue', alpha=0.5, label='Negative')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_ylabel('Anomaly (°C)')
ax2.set_title('SST Anomaly (Reference: 2000-2010)', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# 3. 계절별 트렌드
ax3 = fig.add_subplot(gs[2, 0])
season_names = list(seasonal_trends.keys())
season_slopes = [t['slope'] for t in seasonal_trends.values()]
season_colors = ['green', 'red', 'orange', 'blue']
bars = ax3.bar(season_names, season_slopes, color=season_colors, alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_ylabel('Trend (°C/year)')
ax3.set_title('Seasonal Warming Trends', fontsize=12)
ax3.grid(True, alpha=0.3, axis='y')

# 값 표시
for bar, slope in zip(bars, season_slopes):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{slope:.3f}', ha='center', va='bottom' if height > 0 else 'top')

# 4. 월별 평균 온도
ax4 = fig.add_subplot(gs[2, 1])
monthly_clim = sst_ts.groupby(sst_ts.index.month).mean()
monthly_std = sst_ts.groupby(sst_ts.index.month).std()
months = np.arange(1, 13)
ax4.plot(months, monthly_clim.values, 'o-', color='darkblue', linewidth=2, markersize=8)
ax4.fill_between(months, 
                  monthly_clim.values - monthly_std.values,
                  monthly_clim.values + monthly_std.values,
                  alpha=0.3, color='blue')
ax4.set_xlabel('Month')
ax4.set_ylabel('SST (°C)')
ax4.set_title('Monthly Climatology', fontsize=12)
ax4.set_xticks(months)
ax4.set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
ax4.grid(True, alpha=0.3)

# 5. 극한 이벤트 통계
ax5 = fig.add_subplot(gs[3, :])
if len(heatwaves) > 0:
    # 연도별 이벤트 수 계산
    heatwave_years = pd.to_datetime(heatwaves['start']).year
    yearly_counts = heatwave_years.value_counts().sort_index()
    
    # 모든 연도 포함
    all_years = pd.Series(0, index=range(2000, 2024))
    all_years.update(yearly_counts)
    
    ax5.bar(all_years.index, all_years.values, color='red', alpha=0.7, label='Marine Heatwaves')
    ax5.set_xlabel('Year')
    ax5.set_ylabel('Number of Events')
    ax5.set_title('Extreme Temperature Events Frequency', fontsize=12)
    ax5.legend()
    ax5.grid(True, alpha=0.3, axis='y')

plt.suptitle('East Sea SST Comprehensive Analysis (2000-2023)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(os.path.join(EAST_SEA_CONFIG['output_dir'], 'timeseries_analysis.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 공간 분포 지도
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, figure=fig, hspace=0.15, wspace=0.05)

# 투영법 설정
projection = ccrs.PlateCarree()

# 1-4. 계절별 평균 SST
for idx, (season_name, season_data) in enumerate(seasonal_means.items()):
    ax = fig.add_subplot(gs[idx//2, idx%2], projection=projection)
    
    im = ax.pcolormesh(ds.longitude, ds.latitude, season_data,
                        transform=projection, cmap='RdYlBu_r',
                        vmin=5, vmax=30, shading='auto')
    
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, alpha=0.5)
    ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    ax.set_title(f'{season_name} Mean SST', fontsize=12)
    
    # 컬러바
    cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.8)
    cbar.set_label('°C', fontsize=10)

# 5. 온난화 패턴
ax = fig.add_subplot(gs[:, 2], projection=projection)
im = ax.pcolormesh(ds.longitude, ds.latitude, warming_pattern,
                    transform=projection, cmap='RdBu_r',
                    vmin=-2, vmax=2, shading='auto')

ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.LAND, alpha=0.5)
gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
gl.top_labels = False
ax.set_title('Warming Pattern\n(2019-2023) - (2000-2004)', fontsize=12)

cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.4)
cbar.set_label('Temperature Change (°C)', fontsize=10)

plt.suptitle('East Sea SST Spatial Patterns', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(EAST_SEA_CONFIG['output_dir'], 'spatial_analysis.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

## 6. 분석 결과 저장

In [ ]:
# 분석 결과 종합
analysis_results = {
    'region': EAST_SEA_CONFIG['region_name'],
    'analysis_period': EAST_SEA_CONFIG['analysis_period'],
    'statistics': {
        'mean_sst': float(sst_ts.mean()),
        'std_sst': float(sst_ts.std()),
        'min_sst': float(sst_ts.min()),
        'max_sst': float(sst_ts.max())
    },
    'trends': {
        'overall': {
            'slope_per_year': trend_yearly['slope'],
            'total_warming': trend_yearly['slope'] * 24,  # 24년간 총 변화
            'r_squared': trend_yearly['r_squared'],
            'p_value': trend_yearly['p_value']
        },
        'seasonal': {k: {'slope': v['slope'], 'r_squared': v['r_squared']} 
                    for k, v in seasonal_trends.items()}
    },
    'extreme_events': {
        'heatwaves': {
            'count': len(heatwaves),
            'mean_duration': float(heatwaves['duration'].mean()) if len(heatwaves) > 0 else 0,
            'max_duration': int(heatwaves['duration'].max()) if len(heatwaves) > 0 else 0
        },
        'coldwaves': {
            'count': len(coldwaves),
            'mean_duration': float(coldwaves['duration'].mean()) if len(coldwaves) > 0 else 0,
            'max_duration': int(coldwaves['duration'].max()) if len(coldwaves) > 0 else 0
        }
    },
    'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# JSON으로 저장
cu.save_config(analysis_results, 
               os.path.join(EAST_SEA_CONFIG['output_dir'], 'analysis_results.json'))

# 시계열 데이터 CSV 저장
timeseries_df = pd.DataFrame({
    'date': sst_ts.index,
    'sst_daily': sst_ts.values,
    'sst_anomaly': anomaly_ts.values
})

# 월별 데이터 추가
monthly_df = pd.DataFrame({
    'date': monthly_ts.index,
    'sst_monthly': monthly_ts.values
})

# 병합 및 저장
timeseries_df = timeseries_df.set_index('date')
monthly_df = monthly_df.set_index('date')
combined_df = timeseries_df.join(monthly_df, how='outer')

cu.export_to_csv(combined_df, 
                 os.path.join(EAST_SEA_CONFIG['output_dir'], 'timeseries_data.csv'))

print("\n분석 결과 저장 완료!")

## 7. 분석 요약 및 결론

### 주요 발견사항

1. **장기 온난화 트렌드**
   - 동해 SST는 2000-2023년 기간 동안 뚜렷한 상승 추세를 보임
   - 연평균 온도 상승률과 계절별 차이 확인

2. **계절별 변동성**
   - 여름과 겨울의 온도 차이가 약 15°C
   - 계절별로 다른 온난화 속도 관찰

3. **극한 이벤트**
   - 해양 열파의 빈도와 강도 증가 추세
   - 최근 년도로 갈수록 극한 고온 이벤트 증가

4. **공간적 패턴**
   - 쿠로시오 해류 영향으로 동쪽이 더 따뜻함
   - 위도에 따른 온도 구배 확인

### 추가 분석 제안

1. 엘니뇨/라니냐와의 상관관계 분석
2. 다른 해양 변수(염분, 해류)와의 복합 분석
3. 기후 모델 예측과의 비교
4. 해양 생태계 영향 평가